**EirGrid-Scraping**

This Python script is designed to fetch CO2 data from a specific API. It includes the following key components:

1. **Round Down Time Function (`round_down_time`)**
   - Rounds down the current time to the nearest half-hour, accommodating the data's 30-minute availability intervals.

2. **URL Creation Function (`create_url`)**
   - Constructs the API URL using formatted start and end times and a chosen region.

3. **Time and Region Settings**
   - Sets the current time (`now`), applies the round-down function, and calculates the end time as 24 hours from the current time.
   - Selects a region (example: 'ROI') for data fetching.

4. **API Request**
   - Generates the API URL and performs a GET request to retrieve the data.
   - Stores the response in JSON format.

This script is tailored for accessing time-sensitive environmental data from a designated API endpoint.


In [5]:
from datetime import datetime, timedelta
import requests
from pandas import DataFrame

# data is availble every 30 minutes, so we need to start at the nearest half-hour 
def round_down_time(dt):
    # Round down to the nearest half-hour
    new_minute = 30 if dt.minute >= 30 else 0
    return dt.replace(minute=new_minute, second=0, microsecond=0)

# Function to create the API URL based on the start and end datetimes and the region
def create_url(start_datetime, end_datetime, region):
    start_str = start_datetime.strftime("%Y%m%d%H%M")
    end_str = end_datetime.strftime("%Y%m%d%H%M")
    url = f"https://www.co2.smartgriddashboard.com/api/co2_fc/{start_str}/{end_str}/{region}"
    return url

# Current date and time
now = datetime.now()
# Round down to the nearest half-hour
start_time = round_down_time(now)
# For end time, let's use 24 hours from now
end_time = now + timedelta(days=1)

# Define the region
region = ['ROI', 'NI', 'ALL']

# Create the URL
api_url = create_url(start_time, end_time, region[0])
response = requests.get(api_url)
Rs = response.json()


In [16]:
import pandas as pd
df_carbon_forecast = pd.DataFrame(Rs['Rows'])

# Convert 'EffectiveTime' to datetime and set as index
df_carbon_forecast['EffectiveTime'] = pd.to_datetime(df_carbon_forecast['EffectiveTime'], format='%d-%b-%Y %H:%M:%S')
df_carbon_forecast_indexed = df_carbon_forecast.set_index('EffectiveTime')

# Now you can slice the DataFrame using datetime index
# For example, to access data at '26-JAN-2024 21:30:00'
specific_time_data = df_carbon_forecast_indexed.loc['2024-01-26 21:30:00']

# To access a range of times
time_range_data = df_carbon_forecast_indexed['2024-01-26 21:00:00':'2024-01-26 22:00:00']

# Print to see the results
print(specific_time_data)
print(time_range_data)

FieldName    CO2_INTENSITY_FORECAST
Region                          ROI
Value                      135.0492
Name: 2024-01-26 21:30:00, dtype: object
                                  FieldName Region     Value
EffectiveTime                                               
2024-01-26 21:30:00  CO2_INTENSITY_FORECAST    ROI  135.0492
2024-01-26 22:00:00  CO2_INTENSITY_FORECAST    ROI  132.8175


### **section-for-co2-from-yesterday**

In [27]:
import datetime

# Function to round time to the nearest 15 minutes
def round_time(dt):
    # Round minutes to the nearest 15
    new_minute = (dt.minute // 15) * 15
    return dt.replace(minute=new_minute, second=0, microsecond=0)

# Function to format date in your specific format
def format_date(dt):
    return dt.strftime("%d-%b-%Y").lower() + "+" + dt.strftime("%H%%3A%M")

# Current date and time, rounded to the nearest 15 minutes
now = round_time(datetime.datetime.now())

# Start time (same time yesterday, rounded to the nearest 15 minutes)
yesterday = now - datetime.timedelta(days=1)
startDateTime = format_date(yesterday)

# End time (current time, rounded to the nearest 15 minutes)
endDateTime = format_date(now)

# Print formatted start and end times
print("Start Time:", startDateTime)
print("End Time:", endDateTime)


Start Time: 25-jan-2024+22%3A30
End Time: 26-jan-2024+22%3A30


In [28]:
area = ['CO2Stats', 'generationactual', 'co2emission', 'co2intensity', 'interconnection', 'SnspAll', 'frequency', 'demandactual', 'windactual']
region = ['ROI', 'NI', 'ALL']
Rows = []

url = f'http://smartgriddashboard.eirgrid.com/DashboardService.svc/data?area={area[3]}&region={region[2]}&datefrom={startDateTime}&dateto={endDateTime}'
response = requests.get(url)
Rs = json.loads(response.text)["Rows"]
for row in Rs:
    Rows.append(row)

X = DataFrame(Rows)

In [29]:
X

,EffectiveTime,FieldName,Region,Value
0,25-Jan-2024 22:30:00,CO2_INTENSITY,ALL,147.0
1,25-Jan-2024 22:45:00,CO2_INTENSITY,ALL,148.0
2,25-Jan-2024 23:00:00,CO2_INTENSITY,ALL,149.0
3,25-Jan-2024 23:15:00,CO2_INTENSITY,ALL,150.0
4,25-Jan-2024 23:30:00,CO2_INTENSITY,ALL,151.0
...,...,...,...,...
92,26-Jan-2024 21:30:00,CO2_INTENSITY,ALL,148.0
93,26-Jan-2024 21:45:00,CO2_INTENSITY,ALL,147.0
94,26-Jan-2024 22:00:00,CO2_INTENSITY,ALL,145.0
95,26-Jan-2024 22:15:00,CO2_INTENSITY,ALL,146.0
